In [1]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam


2021-08-24 19:30:48.071008: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-24 19:30:48.071090: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
image_size = (48, 48)
batch_size = 64

train_ds = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255).flow_from_directory(
    "../data/train",
    target_size=image_size,
    batch_size=batch_size,
    color_mode="grayscale",
    class_mode="categorical",
)
val_ds = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255).flow_from_directory(
    "../data/test",
    target_size=image_size,
    batch_size=batch_size,
    color_mode="grayscale",
    class_mode="categorical",
)
#  

Found 15743 images belonging to 7 classes.
Found 3756 images belonging to 7 classes.


In [3]:

# create themodel after adding the layers above each other

model = tf.keras.Sequential()
lay = tf.keras.layers
model.add(lay.Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(48, 48, 1)))
model.add(lay.Conv2D(64, kernel_size=(3, 3), activation="relu"))
model.add(lay.MaxPooling2D(pool_size=(2, 2)))
model.add(lay.Dropout(0.25))

model.add(lay.Conv2D(128, kernel_size=(3, 3), activation="relu"))
model.add(lay.MaxPooling2D(pool_size=(2, 2)))
model.add(lay.Conv2D(128, kernel_size=(3, 3), activation="relu"))
model.add(lay.MaxPooling2D(pool_size=(2, 2)))
model.add(lay.Dropout(0.25))

model.add(lay.Flatten())
model.add(lay.Dense(1024, activation="relu"))
model.add(lay.Dropout(0.5))
model.add(lay.Dense(7, activation="softmax"))



2021-08-24 19:30:55.097323: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-24 19:30:55.097365: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-24 19:30:55.097398: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (abdelmajed-Inspiron-5559): /proc/driver/nvidia/version does not exist
2021-08-24 19:30:55.097863: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
#  7 moods are considered in the applications
emotion_dict = {
    0: "Angry",
    1: "Disgusted",
    2: "Fearful",
    3: "Happy",
    4: "Neutral",
    5: "Sad",
    6: "Surprised",
}

In [5]:
#  compile the Sequential layers in the model
model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(lr=0.0001, decay=1e-6),
    metrics=["accuracy"],
)

/home/abdelmajed/.local/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [6]:
# the machine learning proccess starts here
emotion_model_info = model.fit_generator(
    train_ds,
    steps_per_epoch=500 // 64,
    epochs=1,
    validation_data=val_ds,
    validation_steps=7000 // 64,
)

/home/abdelmajed/.local/lib/python3.8/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2021-08-24 19:30:56.731452: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-08-24 19:30:58.154102: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 17334272 exceeds 10% of free system memory.
2021-08-24 19:30:58.277293: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 31719424 exceeds 10% of free system memory.
2021-08-24 19:30:58.447735: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 13107200 exceeds 10% of free system memory.
2021-08-24 19:30:58.712782: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 13107200 exceeds 10% of free system memory.
2

57/57 [==============================] - 76s 1s/step - loss: 1.2775 - accuracy: 0.4382 - val_loss: 1.1491 - val_accuracy: 0.4723


In [7]:
#  finally save the model as h5 file 
model.save('emotion_model2.h5')
